import libraries

In [ ]:
import streamlink
import time
import cv2 as cv

stream(m3u8) link

In [ ]:
stream_url = "https://manifest.googlevideo.com/api/manifest/hls_variant/expire/1726699152/ei/MALrZpH9FajGi9oP8JeGgQQ/ip/86.107.43.88/id/DnUFAShZKus.1/source/yt_live_broadcast/requiressl/yes/xpc/EgVo2aDSNQ%3D%3D/tx/51257031/txs/51257028%2C51257029%2C51257030%2C51257031%2C51257032/hfr/1/playlist_duration/30/manifest_duration/30/maudio/1/spc/54MbxWGYEv10L5MZxcl1QBLH03orKC2Lm1DTpiyQY2Mqo9MPwbAv0LqcZ42gVds/vprv/1/go/1/rqh/5/pacing/0/nvgoi/1/keepalive/yes/dover/11/itag/0/playlist_type/DVR/sparams/expire%2Cei%2Cip%2Cid%2Csource%2Crequiressl%2Cxpc%2Ctx%2Ctxs%2Chfr%2Cplaylist_duration%2Cmanifest_duration%2Cmaudio%2Cspc%2Cvprv%2Cgo%2Crqh%2Citag%2Cplaylist_type/sig/AJfQdSswRgIhAIlr1yE85xJ5e1k2sQEPQAsoRyX_kXRyWcJU-8igQExGAiEA-PvI-ZgPrvU1ETolGEfhqC0SIG7bN7sDRbpblyL6Umg%3D/file/index.m3u8"
# https://anym3u8player.com/yt/
streams = streamlink.streams(stream_url)
print(streams) 

Saving the live

In [ ]:
if "best" in streams:
    stream = streams["best"]
else:
    print("Stream Not Available.")
    stream = None

if stream:
    start_time = time.time()
    max_duration = 20 * 60 # 10 min

    with open("F:\DA\Projects\Peace_bridge\output3.mp4", "wb") as f:
        for chunk in stream.open():
            f.write(chunk)
            elapsed_time = time.time() - start_time
            if elapsed_time > max_duration:
                print("End.")
                break

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import ffmpeg
import os
import datetime
import pytz
from scipy.signal import find_peaks
import sqlite3

video link

In [ ]:
vid_path = "F:\\DA\\Projects\\Peace_bridge\\output3.mp4"

convert creation time

In [ ]:
def convert_to_canada_time(local_time, local_timezone_str, canada_timezone_str):
    local_timezone = pytz.timezone(local_timezone_str)
    local_time = local_timezone.localize(local_time)
    canada_timezone = pytz.timezone(canada_timezone_str)
    canada_time = local_time.astimezone(canada_timezone)
    return canada_time

def get_file_creation_time(file_path):
    creation_time = os.path.getctime(file_path)
    creation_time_readable = datetime.datetime.fromtimestamp(creation_time)
    date = creation_time_readable.date()
    time = creation_time_readable.time().replace(microsecond=0)
    full_creation_time = datetime.datetime.combine(date, time)
    return full_creation_time

main_time = get_file_creation_time(vid_path)
local_timezone_str = 'Asia/Tehran'
canada_timezone_str = 'America/Toronto'
canada_time = convert_to_canada_time(main_time, local_timezone_str, canada_timezone_str)

video capture

In [ ]:
vid_cv = cv.VideoCapture(vid_path)

if not vid_cv.isOpened():
    print(f"Error: Could not open video file: {vid_path}")
    exit()
ret, frame1 = vid_cv.read()
if not ret or frame1 is None:
    print("Error: Could not read the first frame from the video.")
    exit()

Initialize lists for storing differences and timestamps

In [ ]:
lst = []
timestamps = []

Line1 frame1 processing

In [ ]:
image = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
pts = np.array([[400, 491], [335, 560], [551, 560], [559, 491]])
mask = np.zeros(image.shape[:2], dtype=np.uint8)
cv.fillPoly(mask, [pts], 255)
trapezoid_region = cv.bitwise_and(image, image, mask=mask)
main_mask = trapezoid_region

Other frame processing

In [ ]:
while vid_cv.isOpened():
    ret, frame2 = vid_cv.read()
    if not ret or frame2 is None:
        break
    
    gray = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

    # Line1 processing
    mask = np.zeros(gray.shape[:2], dtype=np.uint8)
    cv.fillPoly(mask, [pts], 255)
    trapezoid_regions = cv.bitwise_and(gray, gray, mask=mask)
    diff = cv.absdiff(main_mask, trapezoid_regions)
    main_mask = trapezoid_regions.copy()
    diff_sum = pow(np.sum(diff), 2)
    lst.append(diff_sum)


    current_time = datetime.timedelta(seconds=frame_count / fps)
    timestamps.append(current_time)
    
    cv.imshow('line1', trapezoid_regions)
    
    if cv.waitKey(30) & 0xFF == ord('q'):
        break

vid_cv.release()
cv.destroyAllWindows()

Find Number Of Cars & Peak Plot

In [ ]:
peaks, _ = find_peaks(lst, height=1e11, distance=100, prominence=1e11)

plt.figure()
plt.plot(np.ravel(lst))
plt.plot(peaks, np.array(lst)[peaks], "x")
plt.title('Line 1 Car Detection')
plt.show()

print(f"Number of peaks detected for Line 1: {len(peaks)}")

Add to DataBase(Sqlite) & Find Peaks

In [ ]:
conn = sqlite3.connect('NewData.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT NOT NULL,
        value INTEGER NOT NULL
    )
''')

cnt_car = 0

if len(timestamps) == len(lst):
    for peak in peaks:
        if peak < len(timestamps):
            cnt_car += 1
            peak_time = timestamps[peak]
            total_seconds = peak_time.total_seconds()
            detection_time = canada_time + datetime.timedelta(seconds=total_seconds)
            formatted_time = detection_time.strftime('%Y-%m-%d %H:%M:%S')
            current_time = formatted_time
            cursor.execute('''
                INSERT INTO records (timestamp, value)
                VALUES (?, ?)
                ''', (current_time, cnt_car))
else:
    print("Error: Length of timestamps list does not match length of lst.")

conn.commit()
conn.close()

In the database, I stored one week's data from Pace Bridge according to the live shows I recorded and their reports in the database.
According to the data, the number of passing cars increases in order (from left to right) on Thursday, Friday, Monday, Tuesday, Wednesday, Saturday and Sunday.